In [188]:
import pandas as pd
import numpy as np
import string
alphabet = string.ascii_uppercase

In [212]:
f=open('quiz13.txt', 'r')
lines_str = f.read().splitlines()

m = len(lines_str)

lines = []

for i in range(len(lines_str)):
    temp_line = []
    for j in range(len(lines_str[i])):
        temp_line.append(lines_str[i][j])
    lines.append(temp_line)

train_location = dict() ##hold the location of each train
train_move = dict()
previous_arrow = dict()
previous_loc = dict()
alive = dict()

##name the trains and initiliaze their locations. The name is the location
count = 0
for i in range(len(lines)):
    for j in range(len(lines[i])):
        character = str(lines[i][j]).replace("'",'').replace('b','')
        if character in ['>', '<', '^', 'v']:
            train_location[str([i,j])] = character
            train_move[str([i,j])] = 0
            previous_arrow[str([i,j])] = ''
            previous_loc[str([i,j])]= ''
            alive[str([i,j])] = True
            count+=1


In [213]:
import ast
for train in sorted([ast.literal_eval(x) for x in list(train_location.keys())]):
    arrow_now = train_location[str(train)].replace("'",'').replace('b','') ##type of arrow
    
    if arrow_now == 'v':
        previous_arrow[str([train[0]-1, train[1]])] = '|'
        previous_loc[str([train[0],train[1]])] = str([train[0]-1, train[1]])
    elif arrow_now == '^':
        previous_arrow[str([train[0]+1, train[1]])] = '|'
        previous_loc[str([train[0],train[1]])] = str([train[0]+1, train[1]])
    elif arrow_now == '>':
        previous_arrow[str([train[0], train[1]-1])] = '-'
        previous_loc[str([train[0],train[1]])] = str([train[0], train[1]-1])
    else:
        previous_arrow[str([train[0], train[1]+1])] = '-'
        previous_loc[str([train[0],train[1]])] = str([train[0], train[1]+1])

# print(train_location)
# print(train_move)
# print(previous_arrow)
# print(previous_loc)

flag = 0
count = 0

while flag == 0:
    if sum(alive.values()) == 1:
        flag = 1
        break

    for train in sorted([ast.literal_eval(x) for x in list(train_location.keys())]):
        arrow_now = train_location[str(train)].replace("'",'').replace('b','') ##type of arrow
        right_ahead_x, right_ahead_y = right_ahead(arrow_now, train[0], train[1])
        arrow_ahead = lines[right_ahead_x][right_ahead_y]
        #print(arrow_now, arrow_ahead)


        if str(train) in previous_loc.keys() and previous_loc[str(train)]!='' and previous_arrow[previous_loc[str(train)]] != '':
            lines[train[0]][train[1]] = previous_arrow[previous_loc[str(train)]] ##put on this track what was here before the arrow moved in here
            ##delete the record for this stored location

        #previous_arrow.pop(str(train))
            ##add the new next arrow
        #previous_arrow[str([right_ahead_x, right_ahead_y])] = arrow_ahead
        previous_arrow[str(train)] = arrow_ahead

        next_loc_x = right_ahead_x
        next_loc_y = right_ahead_y

        previous_loc[str([next_loc_x, next_loc_y])] = str(train) ##the previous of the next location is this location!

        if lines[next_loc_x][next_loc_y] not in ['-', '/', "\\", "+", '|', '']:
            alive[str([next_loc_x, next_loc_y])] = False
            #alive[str(train)] = False
        else:
            alive[str([next_loc_x, next_loc_y])] = True
            
        turn_direction, there_was_an_intersection = get_turn_direction(arrow_ahead, arrow_now, train[0], train[1])
        next_arrow_shape = next_arrow_shape_function(arrow_now, turn_direction)


        ##move the arrow, we already moved whatever was previously on this arrows spot in lines
        lines[next_loc_x][next_loc_y] = next_arrow_shape


        ##save new location
        train_location[str([next_loc_x, next_loc_y])] = next_arrow_shape ##new arrow
        ## delete old location
        train_location.pop(str([train[0], train[1]]))

        ##if there was an intersection
        ##save new move
        if there_was_an_intersection:
            train_move[str([next_loc_x, next_loc_y])] = (train_move[str([train[0], train[1]])]+1)%3 
        #delete old move
            train_move.pop(str([train[0], train[1]]))
        else:
            train_move[str([next_loc_x, next_loc_y])] = train_move[str([train[0], train[1]])]
            train_move.pop(str([train[0], train[1]]))
    count+=1

IndexError: list index out of range

In [58]:
def get_turn_direction(arrow_ahead, arrow_now, i, j):
    
    if arrow_ahead =='+':
        move_number = train_move[str([i,j])]
        
        if move_number==0:
            return 'left',1
        elif move_number ==1:
            return 'straight',1
        else:
            return 'right',1

    elif arrow_ahead == '-' or arrow_ahead == '|':
        return 'straight',0
    
    elif arrow_ahead == '/':
        if arrow_now =='<' or arrow_now =='>':
            return 'left',0
        if arrow_now=='v' or arrow_now =='^':
            return 'right',0
    else: ### arrow_head = '\'
        if arrow_now =='<' or arrow_now=='>':
            return 'right',0
        elif arrow_now == 'v' or arrow_now=='^':
            return 'left',0
        else:
            print(arrow_ahead, arrow_now)
            return None

In [11]:

def next_arrow_shape_function(arrow, turn_direction):
    if turn_direction=='left': ##turn left
        if arrow=='>':
            return '^'
        elif arrow =='^':
            return '<'
        elif arrow == 'v':
            return '>'
        elif arrow =='<':
            return 'v'
        else:
            return None
    
    if turn_direction =='right': ##turn right
        if arrow=='>':
            return 'v'
        elif arrow =='^':
            return '>'
        elif arrow == 'v':
            return '<'
        elif arrow =='<':
            return '^'
        else:
            return None
            
    if turn_direction =='straight': ## go straight
        return arrow
        
    
def right_ahead(arrow, i,j):
    if arrow == '^':
        return i-1,j
    elif arrow =='>':
        return i,j+1
    elif arrow=='<':
        return i,j-1
    elif arrow =='v':
        return i+1,j
    else:
        print(arrow, i, j, 'problem')
        return None

## part 2

In [205]:
def num_carts(lines):
    count_carts = 0
    for i in range(len(lines)):
        for j in range(len(lines[i])):
            if lines[i][j] in ['^', 'v', '<', '>']:
                count_carts+=1
                loc_x = i
                loc_y=j
    if count_carts==1:
        return loc_x, loc_y
    else:
        return None

In [206]:
num_carts(lines)

## test

In [185]:
f=open('day13_test_2.txt', 'r')
lines_str = f.read().splitlines()

m = len(lines_str)

lines = []

for i in range(len(lines_str)):
    temp_line = []
    for j in range(len(lines_str[i])):
        temp_line.append(lines_str[i][j])
    lines.append(temp_line)

train_location = dict() ##hold the location of each train
train_move = dict()
previous_arrow = dict()
previous_loc = dict()

##name the trains and initiliaze their locations. The name is the location
count = 0
for i in range(len(lines)):
    for j in range(len(lines[i])):
        character = str(lines[i][j]).replace("'",'').replace('b','')
        if character in ['>', '<', '^', 'v']:
            train_location[str([i,j])] = character
            train_move[str([i,j])] = 0
            previous_arrow[str([i,j])] = ''
            previous_loc[str([i,j])]= ''
            count+=1


In [186]:
import ast
for train in sorted([ast.literal_eval(x) for x in list(train_location.keys())]):
    arrow_now = train_location[str(train)].replace("'",'').replace('b','') ##type of arrow
    
    if arrow_now == 'v':
        previous_arrow[str([train[0]-1, train[1]])] = '|'
        previous_loc[str([train[0],train[1]])] = str([train[0]-1, train[1]])
    elif arrow_now == '^':
        previous_arrow[str([train[0]+1, train[1]])] = '|'
        previous_loc[str([train[0],train[1]])] = str([train[0]+1, train[1]])
    elif arrow_now == '>':
        previous_arrow[str([train[0], train[1]-1])] = '-'
        previous_loc[str([train[0],train[1]])] = str([train[0], train[1]-1])
    else:
        previous_arrow[str([train[0], train[1]+1])] = '-'
        previous_loc[str([train[0],train[1]])] = str([train[0], train[1]+1])

# print(train_location)
# print(train_move)
# print(previous_arrow)
# print(previous_loc)

flag = 0
count = 0

while flag == 0:
    for train in sorted([ast.literal_eval(x) for x in list(train_location.keys())]):
        arrow_now = train_location[str(train)].replace("'",'').replace('b','') ##type of arrow
        right_ahead_x, right_ahead_y = right_ahead(arrow_now, train[0], train[1])
        arrow_ahead = lines[right_ahead_x][right_ahead_y]
        #print(arrow_now, arrow_ahead)


        if str(train) in previous_loc.keys() and previous_loc[str(train)]!='' and previous_arrow[previous_loc[str(train)]] != '':
            lines[train[0]][train[1]] = previous_arrow[previous_loc[str(train)]] ##put on this track what was here before the arrow moved in here
            ##delete the record for this stored location

        #previous_arrow.pop(str(train))
            ##add the new next arrow
        #previous_arrow[str([right_ahead_x, right_ahead_y])] = arrow_ahead
        previous_arrow[str(train)] = arrow_ahead   

        next_loc_x = right_ahead_x
        next_loc_y = right_ahead_y

        previous_loc[str([next_loc_x, next_loc_y])] = str(train) ##the previous of the next location is this location!

        if lines[next_loc_x][next_loc_y] not in ['-', '/', "\\", "+", '|', '']:
            print(next_loc_x, next_loc_y, count)
            flag = 1
            break

        turn_direction, there_was_an_intersection = get_turn_direction(arrow_ahead, arrow_now, train[0], train[1])
        next_arrow_shape = next_arrow_shape_function(arrow_now, turn_direction)


        ##move the arrow, we already moved whatever was previously on this arrows spot in lines
        lines[next_loc_x][next_loc_y] = next_arrow_shape


        ##save new location
        train_location[str([next_loc_x, next_loc_y])] = next_arrow_shape ##new arrow
        ## delete old location
        train_location.pop(str([train[0], train[1]]))

        ##if there was an intersection
        ##save new move
        if there_was_an_intersection:
            train_move[str([next_loc_x, next_loc_y])] = (train_move[str([train[0], train[1]])]+1)%3 
        #delete old move
            train_move.pop(str([train[0], train[1]]))
        else:
            train_move[str([next_loc_x, next_loc_y])] = train_move[str([train[0], train[1]])]
            train_move.pop(str([train[0], train[1]]))
    count+=1

3 7 13
